In [1]:
import pyranges as pr
import pandas as pd
import numpy as np
from define_helpers import select_rep_site, _define_cryptic_status, select_rep_prox_site, select_rep_five_end
import os


# approach

### Input last exons - selecting representative PAS/last exon:

le_ids can have multiple predictions, need to select a single one.
Strategy:
(starting with novel last exons used for quantification) 
1. Prefer events that match atlas
2. if multiple, select min atlas distance
3. If matches, s





## To-dos:
- fix updating to atlas for -ve strand matching pas
- run selection for all novel 3'UTR extensions
- Implement selection for annotated proximal sites of 3'UTR extensions (i.e. most distal annotated site)


# 

In [2]:
# input novel last exons used to generate combined reference of last exons
novel_le = pr.read_gtf("../data/papa/2023-03-29_papa_i3_cortical_upf1_zanovello_overlap_annotated.gtf")
# dexseq results df (used to extract cryptic events of each class)
dexseq_df = pd.read_csv("../data/papa/2023-05-24_i3_cortical_zanovello.all_datasets.dexseq_apa.results.processed.cleaned.tsv", sep="\t")

cryptics_df = pd.read_csv("../../preprocessing/processed/cryptics_summary_all_events.tsv", sep="\t")

# refeence GTF used to identify novel last exons, quantify vs ref
ref_gtf = pr.read_gtf("../data/reference_filtered.gtf")
# 
tx2le = pd.read_csv("../data/papa/novel_ref_combined.tx2le.tsv", sep="\t")

# last exon quantification regions used as input to Salmon
quant_uniq_le = pr.read_gtf("../data/papa/novel_ref_combined.quant.last_exons.gtf")


info_df = pd.read_csv("../data/papa/novel_ref_combined.info.tsv", sep="\t")


In [3]:
print(quant_uniq_le.columns)
quant_uniq_le.subset(lambda df: ~df.transcript_id.str.startswith("ENST"))

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'transcript_id', 'exon_number',
       'region_rank', 'Start_ref', 'End_ref', 'transcript_id_ref',
       '3p_extension_length', 'event_type', 'ref_gene_id', 'ref_gene_name',
       'le_number', 'le_id'],
      dtype='object')


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24476667,.,+,.,PAPA.NT_19074713_S59.398,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
1,chr1,.,exon,24468985,24476667,.,+,.,PAPA.ctrl_ctrl_3.424,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
2,chr1,.,exon,24468985,24475256,.,+,.,PAPA.TDP43_19065409_S29.438,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
3,chr1,.,exon,24468985,24473773,.,+,.,PAPA.Cont-B_S2.334,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
4,chr1,.,exon,24468985,24474852,.,+,.,PAPA.DZ_curves_1_3.360,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36889,chrY,.,exon,13234576,13234826,.,-,.,PAPA.NT_19074729_S12.25635,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36890,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43-F_S6.20683,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36891,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43_19065413_S19.26168,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36892,chrY,.,exon,13234583,13234826,.,-,.,PAPA.DZ_curves_0_2.21889,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7


In [4]:
dexseq_df.columns

Index(['experiment_name', 'binID', 'groupID', 'featureID', 'exonBaseMean',
       'dispersion', 'stat', 'pvalue', 'padj', 'UsageCoefficient_base',
       'UsageCoefficient_contrast', 'log2fold_contrast_base', 'le_id',
       'gene.qvalue', 'contrast_name', 'gene_id', 'gene_name', 'event_type',
       'annot_status', 'transcript_id', 'chromosome', 'strand', 'start', 'end',
       'mean_PPAU_base', 'mean_PPAU_treatment', 'delta_PPAU_treatment_control',
       'simple_event_type'],
      dtype='object')

In [5]:
novel_le.columns

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'transcript_id', 'event_type',
       '3p_extension_length', 'atlas_filter', 'nearest_atlas_distance', 'Name',
       'motif_filter', 'pas_motifs', 'condition_id', 'le_id', 'gene_name_ref',
       'gene_id_ref', 'mean_PPAU_base', 'mean_PPAU_treatment',
       'delta_PPAU_treatment_control', 'experiment_id', 'Cluster',
       'experiment_count', 'multiple_datasets', 'max_mean_PPAU'],
      dtype='object')

In [6]:
# get le_ids that are novel 3'UTR ectensions
le_id_d3utr = set(dexseq_df[dexseq_df.simple_event_type == "distal_3utr_extension"].le_id)
len(le_id_d3utr)

3369

In [7]:
cryp_le_ids = set(cryptics_df.loc[cryptics_df["simple_event_type"] == "distal_3utr_extension", "le_id"])
cryp_gene_names = set(cryptics_df.loc[cryptics_df["simple_event_type"] == "distal_3utr_extension", "gene_name"])

print(f"Number of cryptic 3'UTR extensions - {len(cryp_le_ids)}")
print(f"Number of cryptic 3'UTR extension-containing genes - {len(cryp_gene_names)}")

Number of cryptic 3'UTR extensions - 104
Number of cryptic 3'UTR extension-containing genes - 104


In [8]:
# add le_id used in downstream analysis
novel_le = novel_le.apply(lambda df: df.merge(tx2le, on="transcript_id", how="inner", suffixes=[None, "_quant"]))

In [10]:
# Select representative PAS for distal 3'UTR extension events (novel)
novel_le_rep_d3utr, rep_choices_d3utr = select_rep_site(novel_le.subset(lambda df: df.le_id_quant.isin(set(le_id_d3utr))), id_col="le_id_quant")


print(f"Number of novel 3'UTR extenstion intervals before selecting representative LEs - {len(novel_le.subset(lambda df: df.le_id_quant.isin(set(le_id_d3utr))))}")
print(f"Number of novel 3'UTR extension intervals after selecting representative LEs - {len(novel_le_rep_d3utr)}")
print({dec: len(ids) for dec, ids in rep_choices_d3utr.items()})

Number of novel 3'UTR extenstion intervals before selecting representative LEs - 15810
Number of novel 3'UTR extension intervals after selecting representative LEs - 3398
{'atlas_1_pred': 1549, 'atlas_max_datasets': 534, 'atlas_max_datasets_shortest': 344, 'motif_1_min': 889, 'motif_shortest_min': 53}


In [11]:
# now need to select representative proximal site for each distal site, since quantification approach lumps together all annotated proximal sites (and compares them to extension)
# since le_ids are annotated sequentially, know that immediately preceding le_number is the partner le_id
d3utr_le_id_split = novel_le_rep_d3utr.le_id.str.split("_", regex=False, expand=True)
print(d3utr_le_id_split)
d3utr_le_id_split[1] = d3utr_le_id_split[1].astype(int).subtract(1)

# reconstruct le_id (corresponding to proximal le_id)
d3utr_prox_le_id = d3utr_le_id_split[0].str.cat(d3utr_le_id_split[1].astype(str), sep="_")
d3utr_prox_le_id

                       0  1
6     ENSG00000001461.17  6
85    ENSG00000023902.14  3
98    ENSG00000031698.13  3
102   ENSG00000033122.21  3
143    ENSG00000053372.5  3
...                  ... ..
1137  ENSG00000129824.16  2
1798   ENSG00000154620.6  2
2091  ENSG00000165246.15  3
60    ENSG00000012817.16  3
3081   ENSG00000230663.1  2

[3398 rows x 2 columns]


6       ENSG00000001461.17_5
85      ENSG00000023902.14_2
98      ENSG00000031698.13_2
102     ENSG00000033122.21_2
143      ENSG00000053372.5_2
                ...         
1137    ENSG00000129824.16_1
1798     ENSG00000154620.6_1
2091    ENSG00000165246.15_2
60      ENSG00000012817.16_2
3081     ENSG00000230663.1_1
Name: 0, Length: 3398, dtype: object

In [12]:
#subset for annotated last exons with 3'end proximal to 3'UTR extensions
quant_uniq_le_d3utr_prox = quant_uniq_le.subset(lambda df: df.le_id.isin(set(d3utr_prox_le_id.values)))
quant_uniq_le_d3utr_prox


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
1,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
2,chr1,.,exon,150158818,150159404,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,568.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
3,chr1,.,exon,150159497,150160065,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,568.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
4,chr1,.,exon,150158818,150159404,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,133.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11573,chrY,.,exon,19703864,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
11574,chrY,.,exon,19705416,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
11575,chrY,.,exon,19705419,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
11576,chrY,.,exon,19705424,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2


In [13]:
# select representative proximal site for partner annotated ALEs (most distal of annotated PAS)
sel_le_d3utr_prox = select_rep_prox_site(quant_uniq_le_d3utr_prox, id_col="le_id")
sel_le_d3utr_prox

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
1,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
2,chr1,.,exon,150159497,150160065,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,568.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
3,chr1,.,exon,109237730,109238182,.,+,.,ENSG00000031698.13,SARS1,...,last,109237373,109237730,ENST00000468588.1,NULL,first_exon_spliced,ENSG00000031698.13,SARS1,2.0,ENSG00000031698.13_2
4,chr1,.,exon,69873466,69875004,.,+,.,ENSG00000033122.21,LRRC7,...,last,69838283,69931506,ENST00000415775.2,NULL,internal_exon_spliced,ENSG00000033122.21,LRRC7,2.0,ENSG00000033122.21_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4289,chrY,.,exon,13705224,13706024,.,+,.,ENSG00000154620.6,TMSB4Y,...,last,13704435,13705224,ENST00000284856.4,NULL,first_exon_spliced,ENSG00000154620.6,TMSB4Y,1.0,ENSG00000154620.6_1
4290,chrY,.,exon,14840412,14845650,.,+,.,ENSG00000165246.15,NLGN4Y,...,last,"14830519,14830519,14830519,14830519","14840412,14840412,14840412,14840412","ENST00000339174.9,ENST00000382868.5,ENST000003...","NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000165246.15,NLGN4Y,2.0,ENSG00000165246.15_2
4291,chrY,.,exon,19703864,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
4292,chrY,.,exon,19703864,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2


In [14]:
# now want to combine the two with minimal info required for making maps
# 3'end coordinates, gene name, le_id, site type, regulation status

# define site_type columns
sel_le_d3utr_prox.site_type = "proximal"
novel_le_rep_d3utr.site_type = "distal"


# background - ns in any dataset (to do this, get list of sig in any dataset)
# get gene IDs with no regulated ALEs in any dataset
# returns pd.Series
ns_gene_ids = (dexseq_df.assign(reg_status=lambda df: np.where(df["padj"].le(0.05), 1, 0))
               .groupby("groupID")
               ["reg_status"]
               .sum()
               .loc[lambda x: x == 0]
             )

# if one isoform of pair is called as cryptic, label both as cryptic
cryp_gene_ids_d3utr = (pd.concat([sel_le_d3utr_prox.as_df()[["le_id", "gene_id"]].drop_duplicates(),
                                  novel_le_rep_d3utr.as_df()[["le_id", "gene_id_ref"]].rename(columns={"gene_id_ref": "gene_id"}).drop_duplicates()],
           ignore_index=True)
           .assign(cryptic_status=lambda df: np.where(df["le_id"].isin(cryp_le_ids), 1, 0))
           # for each gene_name, if any isoforms (prox/distal are called as cryptic, annotated as cryptic)
           .groupby("gene_id")
           ["cryptic_status"].sum()
           .loc[lambda x: x > 0]
)

ns_gene_ids_d3utr = (pd.concat([sel_le_d3utr_prox.as_df()[["le_id", "gene_id"]].drop_duplicates(),
           novel_le_rep_d3utr.as_df()[["le_id", "gene_id_ref"]].rename(columns={"gene_id_ref": "gene_id"}).drop_duplicates()],
           ignore_index=True)
           .assign(cryptic_status=lambda df: np.where(df["gene_id"].isin(ns_gene_ids.index), 1, 0))
           # for each gene_name, if any isoforms (prox/distal are called as cryptic, annotated as cryptic)
           .groupby("gene_id")
           ["cryptic_status"].sum()
           .loc[lambda x: x > 0]
)


print(f"Number of cryptic 3'UTR extension-containing genes - {len(cryp_gene_ids_d3utr)}")
print(f"Number of ns 3'UTR extension-containing genes - {len(ns_gene_ids_d3utr)}")
print(f"Number of cryptic 3'UTR extensions - {len(cryp_le_ids)}")




# sel_le_d3utr_prox
# sel_novel_le_d3utr




Number of cryptic 3'UTR extension-containing genes - 95
Number of ns 3'UTR extension-containing genes - 1624
Number of cryptic 3'UTR extensions - 104


In [25]:
# le_ids of novel extensions with no change in any experiment
ns_le_ids_d3utr = set(dexseq_df.loc[(dexseq_df["groupID"].isin(ns_gene_ids.index)) & (dexseq_df["simple_event_type"] == "distal_3utr_extension"), "le_id"].drop_duplicates())
# now get le_ids of partner proximal les
ns_le_ids_d3utr_prtnr = set(["_".join([le_id.split("_")[0], str(int(le_id.split("_")[1]) - 1)]) for le_id in ns_le_ids_d3utr])

ns_le_ids_d3utr_comb = ns_le_ids_d3utr.union(ns_le_ids_d3utr_prtnr)

In [26]:
# ensure proximal partner of cryptic les are labelled
cryp_le_ids_prtnr = set(["_".join([le_id.split("_")[0], str(int(le_id.split("_")[1]) - 1)]) for le_id in list(cryp_le_ids)])
cryp_le_ids_comb = cryp_le_ids.union(cryp_le_ids_prtnr)

In [27]:
sel_le_d3utr_prox = sel_le_d3utr_prox.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids_comb, ns_le_ids_d3utr_comb, id_col="le_id"))
novel_le_rep_d3utr = novel_le_rep_d3utr.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids_comb, ns_le_ids_d3utr_comb, id_col="le_id_quant"))



# sel_le_d3utr_prox = sel_le_d3utr_prox.assign("cryptic_status", lambda df: pd.Series(np.where(df.gene_id.isin(cryp_gene_ids_d3utr.index), "cryptic", "background"), index=df.index))
# sel_novel_le_d3utr = sel_novel_le_d3utr.assign("cryptic_status", lambda df: pd.Series(np.where(df.gene_id_ref.isin(cryp_gene_ids_d3utr.index), "cryptic", "background"), index=df.index))

In [28]:
novel_le_rep_d3utr.cryptic_status.value_counts()

cryptic_status
NULL          1657
background    1637
cryptic        104
Name: count, dtype: int64

In [30]:
# Construct 'name' string with minimal annotation information
# le_id|gene_name|site_type|cryptic_status

sel_le_d3utr_prox = (sel_le_d3utr_prox
                     .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("ref_gene_name",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         # some ref_gene_name entries have multiple gene names
         lambda df: df["ref_gene_name"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
 .assign("gene_name_common",
                          lambda df: pd.Series(np.where(df["gene_name"].isna(),
                                                        df["ref_gene_name"],
                                                        df["gene_name"])))
 .assign("Name",
                          lambda df: df["le_id"].str.cat(df[["gene_name_common", "site_type", "cryptic_status"]], sep="|"))
                    
 )


novel_le_rep_d3utr = (novel_le_rep_d3utr
                      .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("gene_name_ref",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         lambda df: df["gene_name_ref"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
 .assign("Name",
         lambda df: df["le_id"].str.cat(df[["gene_name_ref", "site_type", "cryptic_status"]], sep="|"))
)


In [31]:
# generate output bed file of proximal and distal PAS
utr3_out_bed = pr.concat([sel_le_d3utr_prox[["Score", "Name"]],
                          novel_le_rep_d3utr[["Score", "Name"]]]
                          )

print(len(utr3_out_bed))
# drop duplicate entries


3855


In [33]:
utr3_out_bed.subset(lambda df: df.Name.isin(utr3_out_bed.drop_duplicate_positions().as_df()["Name"].value_counts().loc[lambda x: x > 1].index))

,Chromosome,Start,End,Score,Strand,Name
0,chr1,155738258,155739010,.,+,ENSG00000132676.16_8|DAP3|proximal|background
1,chr1,155738608,155739010,.,+,ENSG00000132676.16_8|DAP3|proximal|background
2,chr1,182828723,182830384,.,+,ENSG00000135838.14_3|NPL|proximal|background
3,chr1,182828723,182830384,.,+,ENSG00000135838.14_3|NPL|proximal|background
4,chr1,182828723,182830384,.,+,ENSG00000135838.14_3|NPL|proximal|background
...,...,...,...,...,...,...
84,chr22,21469641,21469935,.,+,ENSG00000206140.12_1|TMEM191C|proximal|background
85,chr22,50622753,50623773,.,-,ENSG00000100299.18_1|ARSA|proximal|cryptic
86,chr22,50622753,50623027,.,-,ENSG00000100299.18_1|ARSA|proximal|cryptic
87,chrX,130127449,130129628,.,-,ENSG00000156709.15_3|AIFM1|distal|background


In [34]:
# select representative 5'end coord (the shoretest exon) for a given le_id
utr3_out_bed = select_rep_five_end(utr3_out_bed, id_col = "Name")

In [35]:
utr3_out_bed.subset(lambda df: df.Name.isin(utr3_out_bed.drop_duplicate_positions().as_df()["Name"].value_counts().loc[lambda x: x > 1].index))

,Chromosome,Start,End,Score,Strand,Name
0,chr1,70439100,70445530,.,+,ENSG00000116761.12_2|CTH|distal|background
1,chr1,70439100,70445536,.,+,ENSG00000116761.12_2|CTH|distal|background
2,chr1,44778575,44779122,.,+,ENSG00000142937.12_2|RPS8|distal|background
3,chr1,44778575,44779128,.,+,ENSG00000142937.12_2|RPS8|distal|background
4,chr1,42775208,42781977,.,+,ENSG00000164008.17_2|C1orf50|distal|background
5,chr1,42775208,42781983,.,+,ENSG00000164008.17_2|C1orf50|distal|background
6,chr1,173606034,173610215,.,-,ENSG00000183831.7_3|ANKRD45|distal|background
7,chr1,173606030,173610215,.,-,ENSG00000183831.7_3|ANKRD45|distal|background
8,chr5,168262048,168264155,.,+,ENSG00000145934.17_3|TENM2|distal|background
9,chr5,168262048,168264157,.,+,ENSG00000145934.17_3|TENM2|distal|background


In [36]:
# how many le_ids have distinct coordinates? which collapsing category do they belong to?
from collections import Counter
Counter([k if le_id.split("|")[0] in v else None for le_id in utr3_out_bed.drop_duplicate_positions().as_df()["Name"].value_counts().loc[lambda x: x > 1].index for k,v in rep_choices_d3utr.items()])

Counter({None: 48, 'atlas_1_pred': 7, 'atlas_max_datasets': 5})

In [38]:
# all these events appear to be equidistant from nearest atlas site. In this case pick the most distal as representative (already have function)
select_rep_prox_site(utr3_out_bed, id_col="Name").subset(lambda df: df.Name.isin(utr3_out_bed.drop_duplicate_positions().as_df()["Name"].value_counts().loc[lambda x: x > 1].index))


,Chromosome,Start,End,Score,Strand,Name
0,chr1,70439100,70445536,.,+,ENSG00000116761.12_2|CTH|distal|background
1,chr1,44778575,44779128,.,+,ENSG00000142937.12_2|RPS8|distal|background
2,chr1,42775208,42781983,.,+,ENSG00000164008.17_2|C1orf50|distal|background
3,chr1,173606030,173610215,.,-,ENSG00000183831.7_3|ANKRD45|distal|background
4,chr5,168262048,168264157,.,+,ENSG00000145934.17_3|TENM2|distal|background
5,chr7,100092232,100093133,.,-,ENSG00000166508.18_3|MCM7|distal|background
6,chr11,36229159,36232257,.,+,ENSG00000179241.13_2|LDLRAD3|distal|background
7,chr13,102678226,102679173,.,+,ENSG00000134900.12_3|TPP2|distal|background
8,chr14,54538062,54545302,.,+,ENSG00000100532.13_3|CGRRF1|distal|background
9,chr15,73769469,73770615,.,+,ENSG00000260469.3_2|INSYN1-AS1|distal|background


In [39]:
utr3_out_bed = select_rep_prox_site(utr3_out_bed, id_col="Name")
utr3_out_bed

,Chromosome,Start,End,Score,Strand,Name
0,chr1,24468985,24472976,.,+,ENSG00000001461.17_5|NIPAL3|proximal|background
1,chr1,24468985,24472976,.,+,ENSG00000001461.17_5|NIPAL3|proximal|background
2,chr1,24468985,24476667,.,+,ENSG00000001461.17_6|NIPAL3|distal|background
3,chr1,150159497,150160065,.,+,ENSG00000023902.14_2|PLEKHO1|proximal|background
4,chr1,150158818,150160329,.,+,ENSG00000023902.14_3|PLEKHO1|distal|background
...,...,...,...,...,...,...
3801,chrY,13705224,13706024,.,+,ENSG00000154620.6_1|TMSB4Y|proximal|background
3802,chrY,13705224,13706471,.,+,ENSG00000154620.6_2|TMSB4Y|distal|background
3803,chrY,14840412,14845650,.,+,ENSG00000165246.15_2|NLGN4Y|proximal|background
3804,chrY,14840412,14845666,.,+,ENSG00000165246.15_3|NLGN4Y|distal|background


In [41]:
# drop duplicate entries
utr3_out_bed = utr3_out_bed.apply(lambda df: df.drop_duplicates()).sort()

utr3_out_bed

,Chromosome,Start,End,Score,Strand,Name
0,chr1,1495484,1497848,.,+,ENSG00000160072.20_1|ATAD3B|proximal|background
1,chr1,1495484,1498495,.,+,ENSG00000160072.20_2|ATAD3B|distal|background
2,chr1,3775794,3775982,.,+,ENSG00000235169.11_2|SMIM1|proximal|background
3,chr1,3775794,3776724,.,+,ENSG00000235169.11_3|SMIM1|distal|background
4,chr1,6219235,6220805,.,+,ENSG00000158286.13_3|RNF207|proximal|background
...,...,...,...,...,...,...
3344,chrY,13705224,13706024,.,+,ENSG00000154620.6_1|TMSB4Y|proximal|background
3345,chrY,13705224,13706471,.,+,ENSG00000154620.6_2|TMSB4Y|distal|background
3346,chrY,14840412,14845650,.,+,ENSG00000165246.15_2|NLGN4Y|proximal|background
3347,chrY,14840412,14845666,.,+,ENSG00000165246.15_3|NLGN4Y|distal|background


In [42]:
if not os.path.exists("../processed/iclip_regions"):
    os.makedirs("../processed/iclip_regions")
    
utr3_out_bed.to_bed("../processed/iclip_regions/2023-07-04_papa_cryptic_d3utr.last_exons.bed")
# also make one with just polYA sites
utr3_out_bed.three_end().to_bed("../processed/iclip_regions/2023-07-04_papa_cryptic_d3utr.pas.bed")